# Train

In [88]:
fig, ax = plt.subplots()

FigureCanvasNbAgg()

In [32]:
model = build_model()
trained_model = build_model()
trained_model.set_weights(model.get_weights())
best_model = build_model()
best_model.set_weights(model.get_weights())
#model.summary()

In [92]:
reset_samples()
eval_scores = [eval(trained_model, eval_games)]
all_eval_scores = eval_scores[:]


In [3]:
from Cython.Compiler.Options import get_directive_defaults
directive_defaults = get_directive_defaults()
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [3]:
%%cython -f -I . -L . -L /home/domin/.local/lib/python3.6/site-packages -I /home/domin/.local/lib/python3.6/site-packages  --cplus 

from src.MCTS cimport MCTS, Storage
from src.LookUp cimport LookUp
from gym_watten.envs.watten_env cimport WattenEnv
from IPython.display import clear_output

cpdef void train(WattenEnv env, LookUp model, Storage storage, MCTS mcts):
    cdef int g
    for g in range(1):  
        mcts.mcts_generate(env, model, storage)


        """if hasattr(model, 'fit'):
            p = np.random.permutation(int(number_of_samples))
            if len(p) > BATCH_SIZE * TRAINING_LOOPS:
                p = p[:BATCH_SIZE * TRAINING_LOOPS]
            train_outputs = []
            train_inputs = []
            for i in range(len(sample_outputs)):
                train_outputs.append(sample_outputs[i][p])
            for i in range(len(sample_inputs)):
                train_inputs.append(sample_inputs[i][p])

            trained_model.fit(train_inputs, train_outputs, epochs=1, batch_size=BATCH_SIZE)    

        model.set_weights(trained_model.get_weights())
        rating = compare_given_games(model, best_model, eval_games)
        print("Match: " + str(rating))
        print("Last scores: " + str(eval_scores[::-1]) + "!")
        if rating > 0.5:
            best_model.set_weights(model.get_weights())
            eval_new = calc_exploitability(best_model, eval_games)
            eval_scores.append(eval_new)
            print("Model score: " + str(eval_new) + "!")
            all_eval_scores.append(eval_new)

    plt.plot(all_eval_scores, label='6x75')
    ax.legend(shadow=True)
    plt.show()"""



In [12]:
train(env, model, storage, mcts)

[0.011437416076660156, 0.04109787940979004, 0.015311002731323242]


In [7]:
import pstats, cProfile

cProfile.runctx("train(env, model, storage, mcts)", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

[0, 0, 0]
Thu Mar 22 17:20:44 2018    Profile.prof

         165451 function calls (108117 primitive calls) in 0.070 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
22075/6760    0.026    0.000    0.062    0.000 MCTS.pyx:83(mcts_sample)
74323/32304    0.016    0.000    0.016    0.000 MCTS.pyx:58(calc_q)
     3895    0.003    0.000    0.006    0.000 MCTS.pyx:42(add_state)
     4308    0.003    0.000    0.003    0.000 watten_env.pyx:187(get_state)
       75    0.003    0.000    0.069    0.001 MCTS.pyx:197(mcts_game)
     2015    0.003    0.000    0.003    0.000 LookUp.pyx:18(generate_key)
     4233    0.002    0.000    0.006    0.000 watten_env.pyx:103(step)
     6248    0.002    0.000    0.002    0.000 watten_env.pyx:199(set_state)
    22075    0.002    0.000    0.002    0.000 MCTS.pyx:54(is_state_leaf_node)
      338    0.002    0.000    0.065    0.000 MCTS.pyx:146(mcts_game_step)
     2261    0.001    0.000    0.002    0.0

In [6]:
%lprun -f "mcts.mcts_sample" train(env, model, storage, mcts)

/usr/local/lib/python3.6/dist-packages/line_profiler.py:321: UserWarning: Could not extract a code object for the object 'mcts.mcts_sample'
  profile = LineProfiler(*funcs)


Timer unit: 1e-06 s

In [62]:
model.set_weights(trained_model.get_weights())

In [92]:
eval(models[1], eval_games)

0.690327380952381

In [272]:
all_eval_scores

[0.02589285714285713]

In [251]:
sample_outputs[1][9]

array([1.])

In [209]:
compare_given_games(model, first_model, eval_games)

0.5410714285714285

In [200]:
model.predict_single([train_inputs[0][next_index - 2], train_inputs[1][next_index - 2]])

NameError: name 'train_inputs' is not defined

In [45]:
train_outputs[0][next_index - 2]

array([0.258651  , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.27687292, 0.        , 0.        , 0.        ,
       0.46447608, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [46]:
train_inputs[0][next_index - 2]

array([[[0, 1],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0]],

       [[1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]]])

In [111]:
test_model = build_model()

In [72]:
K.set_value(test_model.optimizer.lr, 0.0001)

In [132]:
test_model.fit([sample_inputs[0][:number_of_samples], sample_inputs[1][:number_of_samples]], [sample_outputs[0][:number_of_samples], sample_outputs[1][:number_of_samples]], epochs=10, batch_size=BATCH_SIZE)  

Epoch 1/10
37764/37764 [==============================] - 1s 27us/step - loss: 0.6618 - dense_14_loss: 2.8052e-04 - dense_15_loss: 0.6615 - dense_14_acc: 0.5164 - dense_15_acc: 0.4518
Epoch 2/10
37764/37764 [==============================] - 1s 24us/step - loss: 0.6619 - dense_14_loss: 2.7928e-04 - dense_15_loss: 0.6616 - dense_14_acc: 0.5180 - dense_15_acc: 0.4519
Epoch 3/10
37764/37764 [==============================] - 1s 24us/step - loss: 0.6618 - dense_14_loss: 2.7612e-04 - dense_15_loss: 0.6616 - dense_14_acc: 0.5203 - dense_15_acc: 0.4518
Epoch 4/10
37764/37764 [==============================] - 1s 24us/step - loss: 0.6618 - dense_14_loss: 2.7579e-04 - dense_15_loss: 0.6615 - dense_14_acc: 0.5171 - dense_15_acc: 0.4513
Epoch 5/10
37764/37764 [==============================] - 1s 25us/step - loss: 0.6618 - dense_14_loss: 2.7352e-04 - dense_15_loss: 0.6615 - dense_14_acc: 0.5153 - dense_15_acc: 0.4513
Epoch 6/10
37764/37764 [==============================] - 1s 26us/step - loss: 0

In [116]:
test_model.predict_single([sample_inputs[0][5], sample_inputs[1][5]])

[array([0.0053017 , 0.0045248 , 0.00548624, 0.00582314, 0.0070843 ,
        0.00583717, 0.00645116, 0.00290738, 0.00513383, 0.0056636 ,
        0.331735  , 0.00607965, 0.0053452 , 0.00601907, 0.33392397,
        0.0070668 , 0.00368037, 0.01111693, 0.00290891, 0.00334794,
        0.00270144, 0.00335149, 0.00295761, 0.00356095, 0.00431994,
        0.00539099, 0.00628497, 0.00557014, 0.3452927 , 0.00649954,
        0.00652416, 0.00288056], dtype=float32),
 array([-0.81022185], dtype=float32)]

In [110]:
sample_outputs[0][10]

37764

In [152]:
sample_outputs[0][10638]

array([0.        , 0.23038077, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.15442885, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.23038077,
       0.        , 0.        , 0.        , 0.23038077, 0.        ,
       0.        , 0.        , 0.        , 0.15442885, 0.        ,
       0.        , 0.        ])

In [97]:
eval(model, eval_cache_input, eval_cache_output)

0.127

In [133]:
model_output = test_model.predict([sample_inputs[0][:number_of_samples], sample_inputs[1][:number_of_samples]])[0]
a = np.argmax(model_output, axis=-1)
k = np.equal(np.take(sample_outputs[0][:number_of_samples], np.argmax(sample_outputs[0][:number_of_samples], axis=-1)), np.take(sample_outputs[0][:number_of_samples], a)).sum()
k / number_of_samples

0.8743247537337147

In [102]:
model.save('modelDense3.h5')

AttributeError: 'LookUp' object has no attribute 'save'

In [82]:
model.load_weights('modelDense2.h5')

In [97]:
compare_given_games(best_model, models[2], eval_games)

IndexError: list index out of range

In [66]:
models.append(best_model)

In [57]:
models = []

In [274]:
first_nn_model = model

In [74]:
eval(best_model, eval_games)

0.690327380952381

In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 8, 2)      0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 64)           0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 68)           0           flatten_1[0][0]                  
                                                                 input_2[0][0]                    
__________